In [1]:
from transformers import AutoTokenizer
import torch
import numpy as np

model_name = 'meta-llama/Meta-Llama-3-8B-Instruct'
tokenizer = AutoTokenizer.from_pretrained(model_name)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [3]:
model_label = model_name.split('/')[-1]

idx = 2

clean_tokens_v1 = torch.load(f'patches/{idx}_clean_tokens.bin', map_location=torch.device('cpu'))
corr_tokens_v1 = torch.load(f'patches/{idx}_corr_tokens.bin', map_location=torch.device('cpu'))

clean_str_tokens_v1 = [tok.replace('Ġ', '')for tok in tokenizer.convert_ids_to_tokens(clean_tokens_v1)]
corr_str_tokens_v1 = [tok.replace('Ġ', '')for tok in tokenizer.convert_ids_to_tokens(corr_tokens_v1)]

clean_tokens_v2 = torch.load(f'patches/{idx}_clean_tokens_v2.bin', map_location=torch.device('cpu'))
corr_tokens_v2 = torch.load(f'patches/{idx}_corr_tokens_v2.bin', map_location=torch.device('cpu'))

clean_str_tokens_v2 = [tok.replace('Ġ', '')for tok in tokenizer.convert_ids_to_tokens(clean_tokens_v2)]
corr_str_tokens_v2 = [tok.replace('Ġ', '')for tok in tokenizer.convert_ids_to_tokens(corr_tokens_v2)]

In [4]:
n_layers = 32
n_heads = 32
n_key_value_heads = 8

In [5]:
def load_patch(idx, comp, patch):
    return torch.load(f'patches/{idx}_s1_AtP_{comp}{patch}.bin')

### Plotting

In [6]:
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plot_atp(atp, str_tokens, component, n_last_tokens=128, val=1, prepend_bos=True):

    xs = [f"{tok} | {i}" for i, tok in enumerate(str_tokens[-n_last_tokens:])]
    
    if component in ['z', 'q', 'result']:
        ys = [f'L{i}H{j}' for i in range(n_layers) for j in range(n_heads)]
    elif component in ['k', 'v']:
        ys = [f'L{i}{component.upper()}{j}' for i in range(n_layers) for j in range(n_key_value_heads)]
    else:
        ys = [f"L{l} {component.upper()}" for l in range(n_layers)]
        
    fig = px.imshow(
        atp[:, -n_last_tokens:].cpu().numpy(), 
        x=xs,
        y=ys,
        color_continuous_scale='RdBu', zmin=-val, zmax=val, aspect='auto'
    )
    
    return fig

In [7]:
def plot_qkv_atp(str_tokens, atp_q, atp_k, atp_v, n_last_tokens=256, val=1, **kwargs):
    fig = make_subplots(rows=3, cols=1, subplot_titles=("Queries", "Keys", "Values"), shared_xaxes=True, vertical_spacing=0.05)
    
    for i, (atp, hook) in enumerate(zip([atp_q, atp_k, atp_v], ['q', 'k', 'v'])):
        plot = plot_atp(atp, str_tokens, component=hook, prepend_bos=False, n_last_tokens=n_last_tokens)
        for trace in plot.data:
            fig.add_trace(trace, row=1+i, col=1)
        
    fig.update_layout(
        coloraxis1=dict(colorscale='RdBu', cmin=-val, cmax=val),
        showlegend=False,
        **kwargs
    )
    
    return fig

def plot_comp_atp(str_tokens, atp_rs, atp_mlp, atp_attn, n_last_tokens=256, val=1, **kwargs):
    fig = make_subplots(rows=3, cols=1, subplot_titles=("Residual Stream", "MLPs", "Attention"), shared_xaxes=True, vertical_spacing=0.05)
    
    for i, (atp, hook) in enumerate(zip([atp_rs, atp_mlp, atp_attn], ['resid_pre', 'mlp_out', 'attn_out'])):
        plot = plot_atp(atp, str_tokens, component=hook, prepend_bos=False, n_last_tokens=n_last_tokens)
        for trace in plot.data:
            fig.add_trace(trace, row=1+i, col=1)
        
    fig.update_layout(
        coloraxis1=dict(colorscale='RdBu', cmin=-val, cmax=val),
        showlegend=False,
        **kwargs
    )
    
    return fig

## Version 1

### Components

In [8]:
atp_rs_v1 = load_patch(idx, 'resid_pre', '') 
atp_mlp_v1 = load_patch(idx, 'mlp_out', '') 
atp_attn_v1 = load_patch(idx, 'attn_out', '')

In [9]:
fig = plot_comp_atp(clean_str_tokens_v1, atp_rs_v1, atp_mlp_v1, atp_attn_v1, val=1)
fig.update_layout(height=1200)

### Heads

In [10]:
atp_h_v1 = load_patch(idx, 'result', '')

In [11]:
fig = px.histogram(atp_h_v1.max(1).values)
fig.add_vline(x=0.05, line_dash="dash", line_color="red")
fig.show()

In [12]:
h_ys = [f'L{i}H{j}' for i in range(n_layers) for j in range(n_heads)]

In [13]:
threshold = 0.2

h_mask = atp_h_v1.abs().max(1).values > threshold
h_ys_ = np.array(h_ys)[h_mask]

len(h_ys_)

18

In [15]:
value_mask = atp_h_v1.abs().argmax(1)[h_mask]
heads_v1 = [y + '+' if s > 0 else y + '-' for y, s in zip(h_ys_, atp_h_v1[h_mask, value_mask])]

In [16]:
n_last_tokens = 256
val = 1
xs = [f"{tok} | {i}" for i, tok in enumerate(clean_str_tokens_v1[-n_last_tokens:])]

fig = px.imshow(
    atp_h_v1[h_mask, -n_last_tokens:].cpu().numpy(), 
    x=xs,
    y=h_ys_,
    color_continuous_scale='RdBu', zmin=-val, zmax=val, aspect='auto'
)

fig.update_layout(
    coloraxis1=dict(colorscale='RdBu', cmin=-val, cmax=val),
    showlegend=False,
    height = 800
)

### QKV

In [17]:
atp_q_v1 = load_patch(idx, 'q', '') 
atp_k_v1 = load_patch(idx, 'k', '') 
atp_v_v1 = load_patch(idx, 'v', '') 

In [18]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("Residual Stream", "MLPs", "Attention"), shared_xaxes=True, vertical_spacing=0.05)
    
for i, atp in enumerate([atp_q_v1, atp_k_v1, atp_v_v1]):
    plot = px.histogram(atp.max(1).values)
    for trace in plot.data:
        fig.add_trace(trace, row=1+i, col=1)

fig.add_vline(x=0.05, line_dash="dash", line_color="red")
fig.update_layout(height=600)
fig.show()

In [20]:
q_ys = [f'L{i}H{j}' for i in range(n_layers) for j in range(n_heads)]
k_ys = [f'L{i}K{j}' for i in range(n_layers) for j in range(n_key_value_heads)]
v_ys = [f'L{i}V{j}' for i in range(n_layers) for j in range(n_key_value_heads)]

In [21]:
threshold = 0.2

q_mask = atp_q_v1.abs().max(1).values > threshold
k_mask = atp_k_v1.abs().max(1).values > threshold
v_mask = atp_v_v1.abs().max(1).values > threshold

In [22]:
q_ys_ = np.array(q_ys)[q_mask]
k_ys_ = np.array(k_ys)[k_mask]
v_ys_ = np.array(v_ys)[v_mask]

len(q_ys_), len(k_ys_), len(v_ys_)

(10, 5, 9)

In [23]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("Queries", "Keys", "Values"), shared_xaxes=True, vertical_spacing=0.05)
n_last_tokens = 256
val = 1
xs = [f"{tok} | {i}" for i, tok in enumerate(clean_str_tokens_v1[-n_last_tokens:])]

for i, (atp, ys, mask) in enumerate(zip([atp_q_v1, atp_k_v1, atp_v_v1], [q_ys_, k_ys_, v_ys_], [q_mask, k_mask, v_mask])):
    plot = px.imshow(
        atp[mask, -n_last_tokens:].cpu().numpy(), 
        x=xs,
        y=ys,
        color_continuous_scale='RdBu', zmin=-val, zmax=val, aspect='auto'
    )
    for trace in plot.data:
        fig.add_trace(trace, row=1+i, col=1)
    
fig.update_layout(
    coloraxis1=dict(colorscale='RdBu', cmin=-val, cmax=val),
    showlegend=False,
    height = 1200
)

## Version 2

### Components

In [24]:
atp_rs_v2 = load_patch(idx, 'resid_pre', '_v2') 
atp_mlp_v2 = load_patch(idx, 'mlp_out', '_v2') 
atp_attn_v2 = load_patch(idx, 'attn_out', '_v2') 

In [25]:
fig = plot_comp_atp(clean_str_tokens_v2, atp_rs_v2, atp_mlp_v2, atp_attn_v2, val=1.5)
fig.update_layout(height=1200)

### Heads

In [26]:
atp_h_v2= load_patch(idx, 'result', '_v2')

In [27]:
fig = px.histogram(atp_h_v2.max(1).values)
fig.add_vline(x=0.05, line_dash="dash", line_color="red")
fig.show()

In [28]:
h_ys = [f'L{i}H{j}' for i in range(n_layers) for j in range(n_heads)]

In [30]:
threshold = 0.2

h_mask = atp_h_v2.abs().max(1).values > threshold
h_ys_ = np.array(h_ys)[h_mask]

len(h_ys_)

18

In [32]:
value_mask = atp_h_v2.abs().argmax(1)[h_mask]
heads_v2 = [y + '+' if s > 0 else y + '-' for y, s in zip(h_ys_, atp_h_v2[h_mask, value_mask])]

In [33]:
n_last_tokens = 256
val = 1
xs = [f"{tok} | {i}" for i, tok in enumerate(clean_str_tokens_v2[-n_last_tokens:])]

fig = px.imshow(
    atp_h_v2[h_mask, -n_last_tokens:].cpu().numpy(), 
    x=xs,
    y=h_ys_,
    color_continuous_scale='RdBu', zmin=-val, zmax=val, aspect='auto'
)

fig.update_layout(
    coloraxis1=dict(colorscale='RdBu', cmin=-val, cmax=val),
    showlegend=False,
    height = 800
)

In [34]:
heads_v2

['L10H30+',
 'L12H30+',
 'L12H31+',
 'L13H30-',
 'L14H31+',
 'L15H31+',
 'L16H31-',
 'L17H31+',
 'L18H31-',
 'L19H31-',
 'L20H31-',
 'L21H31+',
 'L22H31+',
 'L24H31+',
 'L26H31-',
 'L27H31+',
 'L30H31+',
 'L31H31+']

### QKV

In [35]:
atp_q_v2 = load_patch(idx, 'q', '_v2') 
atp_k_v2 = load_patch(idx, 'k', '_v2') 
atp_v_v2 = load_patch(idx, 'v', '_v2') 

In [36]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("Residual Stream", "MLPs", "Attention"), shared_xaxes=True, vertical_spacing=0.05)
    
for i, atp in enumerate([atp_q_v2, atp_k_v2, atp_v_v2]):
    plot = px.histogram(atp.max(1).values)
    for trace in plot.data:
        fig.add_trace(trace, row=1+i, col=1)

fig.add_vline(x=0.05, line_dash="dash", line_color="red")
fig.update_layout(height=600)
fig.show()

In [37]:
q_ys = [f'L{i}H{j}' for i in range(n_layers) for j in range(n_heads)]
k_ys = [f'L{i}K{j}' for i in range(n_layers) for j in range(n_key_value_heads)]
v_ys = [f'L{i}V{j}' for i in range(n_layers) for j in range(n_key_value_heads)]

In [38]:
threshold = 0.1

q_mask = atp_q_v2.abs().max(1).values > threshold
k_mask = atp_k_v2.abs().max(1).values > threshold
v_mask = atp_v_v2.abs().max(1).values > threshold

In [39]:
import numpy as np

q_ys_ = np.array(q_ys)[q_mask]
k_ys_ = np.array(k_ys)[k_mask]
v_ys_ = np.array(v_ys)[v_mask]

len(q_ys_), len(k_ys_), len(v_ys_)

(25, 12, 18)

In [40]:
fig = make_subplots(rows=3, cols=1, subplot_titles=("Queries", "Keys", "Values"), shared_xaxes=True, vertical_spacing=0.05)
n_last_tokens = 256
val = 1
xs = [f"{tok} | {i}" for i, tok in enumerate(clean_str_tokens_v2[-n_last_tokens:])]

for i, (atp, ys, mask) in enumerate(zip([atp_q_v2, atp_k_v2, atp_v_v2], [q_ys_, k_ys_, v_ys_], [q_mask, k_mask, v_mask])):
    plot = px.imshow(
        atp[mask, -n_last_tokens:].cpu().numpy(), 
        x=xs,
        y=ys,
        color_continuous_scale='RdBu', zmin=-val, zmax=val, aspect='auto'
    )
    for trace in plot.data:
        fig.add_trace(trace, row=1+i, col=1)
    
fig.update_layout(
    coloraxis1=dict(colorscale='RdBu', cmin=-val, cmax=val),
    showlegend=False,
    height = 1200
)

In [41]:
heads_v1

['L9H29+',
 'L10H24+',
 'L11H29+',
 'L13H29+',
 'L13H31-',
 'L14H31+',
 'L15H29+',
 'L15H31+',
 'L16H31+',
 'L17H31+',
 'L18H31-',
 'L19H31-',
 'L20H31-',
 'L22H31+',
 'L24H31+',
 'L26H31-',
 'L30H31+',
 'L31H31+']

In [42]:
heads_v2

['L10H30+',
 'L12H30+',
 'L12H31+',
 'L13H30-',
 'L14H31+',
 'L15H31+',
 'L16H31-',
 'L17H31+',
 'L18H31-',
 'L19H31-',
 'L20H31-',
 'L21H31+',
 'L22H31+',
 'L24H31+',
 'L26H31-',
 'L27H31+',
 'L30H31+',
 'L31H31+']